In [1]:
print("All OK")

All OK


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

True

#### Configure the Model

In [3]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
output = model.invoke("hi")
print(output.content)

Hi there! How can I help you today?


In [4]:
#model = ChatGroq(model="gemma2-9b-it")
output = model.invoke("hi")
print(output.content)

Hi there! How can I help you today?


#### Configure the Embedding Model

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
len(embeddings.embed_query("hi"))

768

#### Lets take data embedd it and store in Vector DB

In [6]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader 
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
loader=DirectoryLoader("../data",glob="./*.txt",loader_cls=TextLoader)

In [8]:
docs=loader.load()

In [9]:
docs[0]

Document(metadata={'source': '..\\data\\usa.txt'}, page_content="\n🇺🇸 Overview of the U.S. Economy\nThe United States of America possesses the largest economy in the world in terms of nominal GDP, making it the most powerful economic force globally. It operates under a capitalist mixed economy, where the private sector dominates, but the government plays a significant regulatory and fiscal role. With a population of over 335 million people and a high level of technological advancement, the U.S. economy thrives on a foundation of consumer spending, innovation, global trade, and financial services. It has a highly diversified structure with strong sectors in technology, healthcare, finance, real estate, defense, and agriculture.\n\nU.S. GDP – Size, Composition, and Global Share\nAs of 2024, the United States’ nominal GDP is estimated to be around $28 trillion USD, accounting for approximately 25% of the global economy. It ranks #1 in the world by nominal GDP, far ahead of China (which ra

In [10]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)

In [11]:
new_docs=text_splitter.split_documents(documents=docs)

In [12]:
new_docs

[Document(metadata={'source': '..\\data\\usa.txt'}, page_content='🇺🇸 Overview of the U.S. Economy'),
 Document(metadata={'source': '..\\data\\usa.txt'}, page_content='The United States of America possesses the largest economy in the world in terms of nominal GDP, making it the most powerful economic force globally. It operates under a capitalist mixed economy,'),
 Document(metadata={'source': '..\\data\\usa.txt'}, page_content='It operates under a capitalist mixed economy, where the private sector dominates, but the government plays a significant regulatory and fiscal role. With a population of over 335 million people and a'),
 Document(metadata={'source': '..\\data\\usa.txt'}, page_content='a population of over 335 million people and a high level of technological advancement, the U.S. economy thrives on a foundation of consumer spending, innovation, global trade, and financial services.'),
 Document(metadata={'source': '..\\data\\usa.txt'}, page_content='innovation, global trade, and 

In [13]:
doc_string=[doc.page_content for doc in new_docs]

doc_string

['🇺🇸 Overview of the U.S. Economy',
 'The United States of America possesses the largest economy in the world in terms of nominal GDP, making it the most powerful economic force globally. It operates under a capitalist mixed economy,',
 'It operates under a capitalist mixed economy, where the private sector dominates, but the government plays a significant regulatory and fiscal role. With a population of over 335 million people and a',
 'a population of over 335 million people and a high level of technological advancement, the U.S. economy thrives on a foundation of consumer spending, innovation, global trade, and financial services.',
 'innovation, global trade, and financial services. It has a highly diversified structure with strong sectors in technology, healthcare, finance, real estate, defense, and agriculture.',
 'U.S. GDP – Size, Composition, and Global Share',
 'As of 2024, the United States’ nominal GDP is estimated to be around $28 trillion USD, accounting for approximately 

In [14]:
db = Chroma.from_documents(new_docs, embeddings)

In [15]:
retriever = db.as_retriever(search_kwargs={"k": 3})

In [16]:
retriever.invoke("What is industrial growth in USA?")

[Document(metadata={'source': '..\\data\\usa.txt'}, page_content='The United States of America possesses the largest economy in the world in terms of nominal GDP, making it the most powerful economic force globally. It operates under a capitalist mixed economy,'),
 Document(metadata={'source': '..\\data\\usa.txt'}, page_content='The U.S. maintains its GDP growth through strong innovation, entrepreneurship, and investment in R&D. With companies like Apple, Google, Amazon, Microsoft, and Tesla leading global markets, the U.S.'),
 Document(metadata={'source': '..\\data\\usa.txt'}, page_content="As a global innovation hub, the U.S. attracts billions in foreign investment and hosts many of the world's largest and most valuable companies. Its exports include high-tech machinery, aircraft,")]

#### Creation of Pydantic Class

In [17]:
import operator
from typing import List
from pydantic import BaseModel , Field
from langchain.prompts import PromptTemplate
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph,END

In [18]:
class TopicSelectionParser(BaseModel):
    Topic:str=Field(description="selected topic")
    Reasoning:str=Field(description='Reasoning behind topic selection')

In [19]:
from langchain.output_parsers import PydanticOutputParser

In [20]:
parser=PydanticOutputParser(pydantic_object=TopicSelectionParser)

In [21]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"Topic": {"description": "selected topic", "title": "Topic", "type": "string"}, "Reasoning": {"description": "Reasoning behind topic selection", "title": "Reasoning", "type": "string"}}, "required": ["Topic", "Reasoning"]}\n```'

In [22]:
class Agentstate(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [23]:
Agentstate = {}

In [24]:
Agentstate["messages"] = []

In [25]:
Agentstate

{'messages': []}

In [26]:
Agentstate["messages"].append("hi how are you?")

In [27]:
Agentstate

{'messages': ['hi how are you?']}

In [28]:
Agentstate["messages"].append("what are you doing?")

In [29]:
Agentstate

{'messages': ['hi how are you?', 'what are you doing?']}

In [30]:
Agentstate["messages"].append("I hope everything is fine!")

In [31]:
Agentstate

{'messages': ['hi how are you?',
  'what are you doing?',
  'I hope everything is fine!']}

In [32]:
Agentstate["messages"][-1]

'I hope everything is fine!'

In [33]:
Agentstate["messages"][0]

'hi how are you?'

In [34]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [35]:
def function_1(state:AgentState):
    
    question=state["messages"][-1]
    
    print("Question",question)
    
    template="""
    Your task is to classify the given user query into one of the following categories: [USA,Not Related]. 
    Only respond with the category name and nothing else.

    User query: {question}
    {format_instructions}
    """
    
    prompt= PromptTemplate(
        template=template,
        input_variable=["question"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    
    
    chain= prompt | model | parser
    
    response = chain.invoke({"question":question})
    
    print("Parsed response:", response)
    
    return {"messages": [response.Topic]}

In [36]:
state1={"messages":["what is a today weather?"]}

In [37]:
state2={"messages":["what is a GDP of usa??"]}

In [41]:
function_1(state1)

Question what is a today weather?
Parsed response: Topic='Not Related' Reasoning="The query is a general weather question and doesn't specify a location, making it unrelated to the USA."


{'messages': ['Not Related']}

In [40]:
function_1(state2)

Question what is a GDP of usa??
Parsed response: Topic='USA' Reasoning='The query explicitly asks for the GDP of the USA.'


{'messages': ['USA']}

In [42]:
class TopicSelectionParser(BaseModel):
    Topic:str=Field(description="selected topic")
    Reasoning:str=Field(description='Reasoning behind topic selection')

In [43]:
def router(state:AgentState):
    print("-> ROUTER ->")
    
    last_message=state["messages"][-1]
    print("last_message:", last_message)
    
    if "usa" in last_message.lower():
        return "RAG Call"
    else:
        return "LLM Call"

In [44]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [45]:
# RAG Function
def function_2(state:AgentState):
    print("-> RAG Call ->")
    
    question = state["messages"][0]
    
    prompt=PromptTemplate(
        template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:""",
        
        input_variables=['context', 'question']
    )
    
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
    result = rag_chain.invoke(question)
    return  {"messages": [result]}

In [46]:
# LLM Function
def function_3(state:AgentState):
    print("-> LLM Call ->")
    question = state["messages"][0]
    
    # Normal LLM call
    complete_query = "Anwer the follow question with you knowledge of the real world. Following is the user question: " + question
    response = model.invoke(complete_query)
    return {"messages": [response.content]}

#### Create StateGraph using Langgraph

In [47]:
from langgraph.graph import StateGraph,END

In [48]:
workflow=StateGraph(AgentState)

In [49]:
workflow.add_node("Supervisor",function_1)

In [50]:
workflow.add_node("RAG",function_2)

In [51]:
workflow.add_node("LLM",function_3)

In [52]:
workflow.set_entry_point("Supervisor")

In [53]:
workflow.add_conditional_edges(
    "Supervisor",
    router,
    {
        "RAG Call": "RAG",
        "LLM Call": "LLM",
    }
)

In [54]:
workflow.add_edge("RAG",END)
workflow.add_edge("LLM",END)

In [55]:
app = workflow.compile()

In [56]:
state={"messages":["hi"]}

In [58]:
app.invoke(state)

Question hi
Parsed response: Topic='Not Related' Reasoning="The query 'hi' is a greeting and does not contain any information related to the USA."
-> ROUTER ->
last_message: Not Related
-> LLM Call ->


{'messages': ['hi', 'Not Related', 'Hi there!']}

In [59]:
state={"messages":["what is a gdp of usa?"]}

In [77]:
app.invoke(state)

Question what is a gdp of usa?
Parsed response: Topic='USA' Reasoning='The query explicitly asks for the GDP of the USA.'
-> ROUTER ->
last_message: USA
-> RAG Call ->


{'messages': ['what is a gdp of usa?',
  'USA',
  "The nominal GDP of the USA is approximately $28 trillion USD as of 2024.  This makes it the world's largest economy by nominal GDP.  It accounts for about 25% of the global economy."]}

In [64]:
state1={"messages":["can you tell me the industrial growth of world's most powerful economy?"]}

In [65]:
state2={"messages":["can you tell me the industrial growth of world's poor economy?"]}

In [70]:
result=app.invoke(state1)

Question can you tell me the industrial growth of world's most powerful economy?
Parsed response: Topic='USA' Reasoning="The query asks about the industrial growth of the world's most powerful economy, which is generally considered to be the USA."
-> ROUTER ->
last_message: USA
-> RAG Call ->


In [71]:
result["messages"][-1]

"The U.S. has the world's largest economy, with a nominal GDP of $28 trillion.  It's considered the engine of global growth due to innovation and financial strength.  Further specifics on industrial growth rates are not provided in the context."

In [74]:
result=app.invoke(state2)

Question can you tell me the industrial growth of world's poor economy?
Parsed response: Topic='Not Related' Reasoning="The query asks about the industrial growth of the world's poor economies, which is a global issue not specifically related to the USA."
-> ROUTER ->
last_message: Not Related
-> LLM Call ->


In [75]:
result["messages"][-1]

'There\'s no single, easily quantifiable answer to "the industrial growth of the world\'s poor economies."  The term "world\'s poor economies" itself is imprecise and encompasses a vast and diverse group of nations with wildly varying levels of development, industrialization, and economic structures.  Further complicating things is the lack of completely reliable and consistent data across all these countries.\n\nHowever, we can make some general observations:\n\n* **Uneven Growth:** Industrial growth in poorer economies is highly uneven. Some countries have experienced significant industrial expansion, often driven by specific sectors like textiles, manufacturing of basic goods, or resource extraction.  Others have lagged considerably, often due to factors like political instability, lack of infrastructure, limited access to capital, corruption, and dependence on primary commodity exports.\n\n* **Shifting Manufacturing Landscape:**  A significant portion of manufacturing previously co